In [18]:
from time import time
import collections

In [61]:
def def_onlyCode(allLines):
    copyline = allLines.copy()
    newLines=[]
    lines=iter(allLines)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    blankLine = lambda x : x=='\n'
    commWildcard = lambda x:x[0]=='#'
    commOneline = lambda x:(initComment(x) and endComment(x[3:]))
    initComment = lambda x:(x[:3]=='"""')
    endComment = lambda x:(x[-4:-1]=='"""')
    
    for line in lines:
        citer=citer+1
        newLine = withoutSpace(line)
        if commWildcard(newLine):
            continue
        if blankLine(newLine):
            continue
        if commOneline(newLine):
            continue
        if initComment(newLine):
            line = next(lines)
            for i in range(citer,len(copyline)):
                newLine = withoutSpace(line)
                endLine = citer+1==len(copyline)
                line = next(lines)
                if endComment(newLine) or endLine:
                    break
                else:
                    citer=citer+1
        newLines.append(line)   
    return newLines

In [62]:
def depFor(archivo):
    newLineas=archivo.copy()
    lineas=iter(archivo)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    initFor = lambda x:(x[:3]=='for')
    endFor = lambda x:(x[-2:-1]==':')
    countSpace = lambda x:(x.index(x.replace(' ','')[0]))
    arrMax=[0]
    arr=[]
    for linea in lineas:
        citer=citer+1
        wordFor='for'
        wordEnd=':'
        space = countSpace(linea)
        patronFor=((initFor(withoutSpace(linea))) and (endFor(withoutSpace(linea))))
        if patronFor:
            newLines=[]
            for i in range(citer,len(newLineas)):
                endLine = citer+1==len(newLineas)
                forSpace = False
                if not endLine:
                    nextspaceLine = countSpace(newLineas[citer])
                    forSpace = nextspaceLine<=space
                else:
                    try:
                        newLines.append(next(lineas))
                    except:
                        pass
                if forSpace or endLine:
                    value,arr=depFor(newLines)
                    arrMax.append(value+1)
                    break
                else:
                    try:
                        newLines.append(next(lineas))
                        citer=citer+1
                    except:
                        pass
    return(max(arrMax),arrMax)

In [63]:
def librerias(lines):
    setlibrerias=set()
    wordImport= 'import'
    wordFrom= 'from'
    notElement={'import', 'from', ''}
    for line in lines:
        inFrom=(wordFrom in line)
        if inFrom:
            arrlibr = line[:line.index('import')].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
        inImport=(wordImport in line)
        if inImport:
            arrlibr =  line[:-1].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
    return setlibrerias-notElement

In [64]:
def countDef(lines):
    wordDef='def'
    wordEnd=':'
    arrArg=[]
    for line in lines:
        if wordDef in line and wordEnd in line:
            argini=line.find("(")+1
            argend=line.find(")")
            arg=line[argini:argend].split(',')
            arrArg.append(len(arg))
    return arrArg

In [71]:
def duplicateLine(arrlines):
    newGroup=[]
    withoutSpace = lambda x : x.replace(' ','')
    lines=list(map(withoutSpace,arrlines))
    unionLine=lambda x : x.replace('[','').replace(',','').replace(']','')
    isDuplicate= lambda x: True if x>=2 else False
    for i,line in enumerate(lines):
        newGroup.append(str(unionLine(str(lines[i:i+4]))))
    repetidos=collections.Counter(newGroup)
    duplicado = [isDuplicate(repetidos[key]) for key in repetidos]
    evalrepetidos = str(duplicado).replace(',',' or ')
    return (eval(evalrepetidos)[0])

In [73]:
start_time = time()

#open file
file = open('bar_chart.py', 'r')
lines = file.readlines()

##Prosesing file

#Lines of code
linesCode = def_onlyCode(lines)
print(len(linesCode))

countFor = depFor(linesCode)
print(countFor[1][1:])


libreriasSet = librerias(linesCode)
print(libreriasSet)

countArgdef = countDef(linesCode)
print(countArgdef)

isDupplicate = duplicateLine(linesCode)
print(isDupplicate)

#close file
file.close()


elapsed_time = time() - start_time
print(elapsed_time)

89
[1, 1, 1, 1]
{'sys', 'decimal', 'math', 'collections', 'optparse'}
[3, 2, 3]
True
0.005982637405395508
